# Sum Text numbers

Code Created by Luis Enrique Acevedo Galicia

Date: 2019-02-07

Taken and adapted from Python Cookbook 3rd -edition

You can perform arithmetic operations using numbers written with letters. The same for the operators. The code accepts both numbers and letters (upper or lower cases)

In [35]:
import re
import collections

In [151]:
# Token specification
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)'
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
ZERO = r'(?P<ZERO>zero)'
ONE = r'(?P<ONE>one)'
TWO = r'(?P<TWO>two)'
THREE = r'(?P<THREE>three)'
FOUR = r'(?P<FOUR>four)'
FIVE = r'(?P<FIVE>five)'
SIX = r'(?P<SIX>six)'
SEVEN = r'(?P<SEVEN>seven)'
EIGHT = r'(?P<EIGHT>eight)'
NINE = r'(?P<NINE>nine)'
WS = r'(?P<WS>\s+)'
PLUSs = r'(?P<PLUSs>plus)'
MINUSs = r'(?P<MINUSs>minus)'
TIMESs = r'(?P<TIMESs>times)'
DIVIDEs = r'(?P<DIVIDEs>divide)'
master_pat = re.compile('|'.join([NUM, PLUS, MINUS, TIMES, DIVIDE, LPAREN, RPAREN,ZERO,ONE,TWO,THREE, FOUR, FIVE, SIX, SEVEN, EIGHT, NINE, PLUSs, MINUSs, TIMESs, DIVIDEs, WS]))

In [152]:
from collections import namedtuple
Token = collections.namedtuple('Token', ['type','value'])

In [154]:
def generate_tokens(text):
    text=text.lower()
    scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):
        tok = Token(m.lastgroup, m.group())
        if tok.type=='ZERO':
            tok = Token('NUM', '0')
        if tok.type=='ONE':
            tok = Token('NUM', '1')
        if tok.type=='TWO':
            tok = Token('NUM', '2')
        if tok.type=='THREE':
            tok = Token('NUM', '3')
        if tok.type=='FOUR':
            tok = Token('NUM', '4')
        if tok.type=='FIVE':
            tok = Token('NUM', '5')
        if tok.type=='SIX':
            tok = Token('NUM', '6')
        if tok.type=='SEVEN':
            tok = Token('NUM', '7')
        if tok.type=='EIGHT':
            tok = Token('NUM', '8')    
        if tok.type=='NINE':
            tok = Token('NUM', '9')
        if tok.type=='PLUSs':
            tok = Token('PLUS', '+')
        if tok.type=='MINUSs':
            tok = Token('MINUS', '-')
        if tok.type=='TIMESs':
            tok = Token('TIMES', '*')
        if tok.type=='DIVIDEs':
            tok = Token('DIVIDE', '/')
            #print(tok)
        if tok.type != 'WS':
            yield tok
        
def matchcase(word):
    return word.lower()



In [155]:
# Parser
class ExpressionEvaluator:
    
    def parse(self,text):
        self.tokens = generate_tokens(text)
        self.tok = None
        # Last symbol consumed
        self.nexttok = None
        # Next symbol tokenized
        self._advance()
        # Load first lookahead token
        return self.expr()
    
    def _advance(self):
        'Advance one token ahead'
        self.tok, self.nexttok = self.nexttok, next(self.tokens, None)
    
    def _accept(self,toktype):
        'Test and consume the next token if it matches toktype'
        if self.nexttok and self.nexttok.type == toktype:
            self._advance()
            return True
        else:
            return False
    def _expect(self,toktype):
        'Consume next token if it matches toktype or raise SyntaxError'
        if not self._accept(toktype):
            raise SyntaxError('Expected ' + toktype)
            
    def expr(self):
        "expression ::= term { ('+'|'-') term }*"
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval += right
            elif op == 'MINUS':
                exprval -= right
        return exprval
        
    def term(self):
        "term ::= factor { ('*'|'/') factor }*"
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval *= right
            elif op == 'DIVIDE':
                termval /= right
        return termval   
    
    def factor(self):
        "factor ::= NUM | ( expr )"
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')

In [184]:
e= ExpressionEvaluator()

#Operation with numbers
print("Operation with numbers: " + str(e.parse('1+2*(3+9)/5')))

#Operation with upper letters
print("Operation with upper letters: " + str(e.parse('ONE+TWO*(THREE+NINE)/FIVE')))

#Operation with upper and lower letters
print("Operation with upper and lower letters: " + str(e.parse('ONE+two*(THREE+NINE)/five')))

#Operation with only with upper letters
print("Operation with only with upper letters: " + str(e.parse('ONE PLUS TWO TIMES (THREE PLUS NINE) DIVIDE FIVE')))


#Operation with only with upper and lower letters
print("Operation with only with upper and lower letters: " + str(e.parse('ONE plus TWO times (THREE PLUS nine) DIVIDE FIVE')))

#Operation with only with letters and numbers
print("Operation with only with letters and numbers: " + str(e.parse('1 plus TWO * (3 PLUS nine) DIVIDE FivE')))

Operation with numbers: 5.8
Operation with upper letters: 5.8
Operation with upper and lower letters: 5.8
Operation with only with upper letters: 5.8
Operation with only with upper and lower letters: 5.8
Operation with only with letters and numbers: 5.8
